In [1]:
import os

from typing import Type, Dict, List, Tuple, Optional

import matplotlib.pyplot as plt
import torch
import pyro
import seaborn as sns
import torch
import contextlib

from torch.utils.data import DataLoader

import pyro

from cities.modeling.simple_linear import SimpleLinear
from cities.modeling.add_categorical_interactions import AddCategoricalInteractions
from cities.modeling.add_causal_layer import AddCausalLayer 
from cities.modeling.svi_inference import run_svi_inference
from cities.utils.data_loader import select_from_data
from pyro.infer import Predictive

import contextlib


smoke_test = "CI" in os.environ
n = 20 if smoke_test else 2000
num_samples = 10 if smoke_test else 1000
n_steps = 10 if smoke_test else 1000

from cities.utils.data_grabber import find_repo_root

root = find_repo_root()

In [2]:
zoning_data_path = os.path.join(root, "data/minneapolis/processed/zoning_dataset.pt")

zoning_dataset_read = torch.load(zoning_data_path)

zoning_loader = DataLoader(
    zoning_dataset_read, batch_size=len(zoning_dataset_read), shuffle=True
)

data = next(iter(zoning_loader))


print(data['categorical'].keys())
print(data['continuous'].keys())


minimal_kwargs = {  
    "categorical": ["past_reform"],
    "continuous": {"parcel_area"},
    "outcome": "housing_units",
}

minimal_subset = select_from_data(data, minimal_kwargs)

# will want to add zone_id -> parcel_area and zone_id -> housing_units
expanded_kwargs = {  
    "categorical": ["zone_id", "past_reform"],
    "continuous": {"parcel_area"},
    "outcome": "housing_units",
}

expanded_subset = select_from_data(data, expanded_kwargs)



dict_keys(['parcel_id', 'neighborhood_id', 'ward_id', 'zone_id', 'limit_id', 'past_reform', 'year', 'month'])
dict_keys(['parcel_area', 'car_parking', 'housing_units', 'parcel_area_original', 'car_parking_original', 'housing_units_original'])


In [3]:
base_model = SimpleLinear(**minimal_subset)

print(minimal_subset['categorical'].keys())
print(minimal_subset['continuous'].keys())

dict_keys(['past_reform'])
dict_keys(['parcel_area'])


In [5]:
with AddCausalLayer(base_model,
        model_kwargs = minimal_kwargs,
        dataset = data,
        causal_layer={"parcel_area": ["zone_id", 'neighborhood_id'], "housing_units": ["zone_id"]}):
    base_model(**minimal_subset)

TypeError: AddCausalLayer() got an unexpected keyword argument 'model_kwargs'

In [3]:
model = SimpleLinear(**subset)

with AddCategoricalInteractions(model, 
            interaction_tuples = [("zone_id", "past_reform")]):
    with pyro.poutine.trace() as tr:
        model(**subset)
    
print(tr.trace.nodes.keys())

odict_keys(['sigma', 'data', 'weights_categorical_zone_id_past_reform', 'bias_continuous', 'weight_continuous', 'mean_outcome_prediction', 'outcome_observed'])
